In [3]:
#imports
import requests
import pandas as pd
import numpy as np

In [4]:
#api key for guardian loaded from evniromental variable
GUARDIAN_KEY = %env GUARDIAN_KEY

#guardian news api endpoint sorted by newest
url = 'https://content.guardianapis.com/news?order-by=newest&api-key=test'

In [7]:
#send a reguest
r = requests.get(url)
r.status_code

200

In [12]:
#read json response to news variable, and then to dataframe
news = r.json()
news = pd.DataFrame(news['response']['results'])

In [14]:
#links for acticles are in webUrl column
url_list = news.webUrl

In [17]:
url_list[0]

'https://www.theguardian.com/news/2023/feb/25/for-the-record'